In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


import plotly.express as px




In [137]:
df.columns

Index(['business_id', 'date', 'review_id', 'stars', 'text', 'type', 'user_id',
       'cool', 'useful', 'funny', 'sentiment'],
      dtype='object')

In [138]:
df = pd.read_csv('yelp.csv')
df.head(1)

X = df[['text']]
y = df[['stars', 'cool', 'useful', 'funny']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



In [139]:
fig = px.histogram(df, x='stars')
fig.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width = 1.5)
#fig.update_layout(
#                    width =1200,
 #                   height = 600
  #                  )

fig.show()

In [140]:
#i have some issues with installing wordcloud therefore i couldnt include it, ill try to fix it in the next days

In [141]:
df['sentiment'] = df['stars'].apply(lambda stars: +1 if stars > 3 else -1)
df['funny'] = df['funny'].apply(lambda funny: +1 if funny > 3 else -1)
df['useful'] = df['useful'].apply(lambda useful: +1 if useful > 3 else -1)
df['cool'] = df['cool'].apply(lambda cool: +1 if cool >3 else -1)
df.sample(5)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny,sentiment
6126,oMycF1cQgR1UVkafXfof7A,2010-04-01,H3hTMeoiwtpfZ4ctjO0Urw,2,I have read several raving reviews of this pla...,review,SGXahHRReksq3m_Zk9V20w,-1,-1,-1,-1
7709,gUt-pPUpOVVhaCFC8-E4yQ,2011-05-18,i_djrvoDoOSb-WM6UsMNYg,4,This is a standard place to celebrate Birthday...,review,5gmnzvjYprzK21Q5awYF_Q,-1,-1,-1,1
7457,EWMwV5V9BxNs_U6nNVMeqw,2012-01-10,lTpCc1QyuzcROvgz-FxKAQ,5,"Along with Bliss, FEZ is another one of my fav...",review,nBFyVu9NZ9zhY49LRoULKA,-1,-1,-1,1
9524,wNUea3IXZWD63bbOQaOH-g,2012-11-12,bKwkVC0g_ooNvhyjSIsL5A,2,I'm not impressed with their medium to high pr...,review,ZPk_wcW7Xot6LngEUj_F6g,-1,-1,-1,-1
485,FKlr2XmPA7G5MlMFpka1OQ,2010-10-16,wHEBxpIC34f6NiZc578vfg,3,I don't like mega-chain pizza. I don't like D...,review,VcN9i7sPrGSOn4BnpOJ9tQ,-1,-1,-1,-1


In [143]:
def cleaning(text):
    clean = ''.join(u for u in text if u not in (',','!','.','?',':',';','"','-','\''))
    return clean

df['text'] = df['text'].apply(cleaning)
df.sample(4)




,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny,sentiment
8075,Jhl-pgZ_ArUOIOSnI4MWpQ,2011-10-07,2ennhxbzgVB4WZwBj8nlyQ,5,I see Jenny Strebe who cuts and Veronica Peebl...,review,iAD9GVUI8m5KaHgi1CnJBQ,-1,-1,-1,1
6286,oIkwj10HmUiognO68j7jOg,2011-07-15,xPwM_eTRNfnh-yeCx2dKUw,4,I have never eaten a regular meal here I usual...,review,-494b8jvjlI07v9HheHhrw,-1,-1,-1,1
9994,L3BSpFvxcNf3T_teitgt6A,2012-03-19,0nxb1gIGFgk3WbC5zwhKZg,5,Lets seewhat is there NOT to like about Surpri...,review,OzOZv-Knlw3oz9K5Kh5S6A,-1,-1,-1,1
9553,hqVbmZNLTdR2y7G7oJu9vg,2011-08-11,Urk4JgrVfqV6-wNP6BLnQw,4,Just went back here and had another fabulous m...,review,lCVLT95VUnlZfboVoZJCZA,-1,-1,-1,1


In [144]:
dfnew = df[['text','sentiment','cool','useful','funny']]
dfnew['text'] = dfnew['text'].str.lower()



C:\Users\Matzo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [145]:
#test and train data
df_train = dfnew.sample(frac=0.8)
df_test = dfnew.drop(df_train.index)

In [153]:
model = CountVectorizer()

train_set = model.fit_transform(df_train['text'])
test_set = model.transform(df_test['text'])

t = pd.DataFrame(data = train_set.toarray(), columns = model.get_feature_names())
t.sample(3)


,00,007,00s,01,0107,02,03342,039,04,05,...,zupas,zur,zuzu,zuzus,zweigel,zwiebelkräuter,zzzzzzzzzzzzzzzzz,éclairs,école,ém
4921,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3980,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7715,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
h = t.sum().sort_values(ascending=False)

In [160]:
h[50:70]

really     2705
our        2674
about      2666
some       2648
service    2478
would      2383
been       2383
an         2364
what       2351
back       2331
your       2312
can        2292
which      2288
more       2156
dont       2106
only       2053
will       2044
also       2039
by         1961
no         1882
dtype: int64

In [161]:
lg = LogisticRegression(solver='lbfgs', max_iter=8000)
#traintest
X_train = train_set
X_test = test_set
y_train = df_train['sentiment']
y_test = df_test['sentiment']

lg.fit(X_train, y_train)

LogisticRegression(max_iter=8000)

In [162]:
predictions = lg.predict(X_test)


In [164]:
#accuracy
a = np.asarray(y_test)
confusion_matrix(predictions, y_test)


array([[ 438,  146],
       [ 185, 1231]], dtype=int64)

In [175]:
print(classification_report(predictions, y_test))

              precision    recall  f1-score   support

          -1       0.70      0.75      0.73       584
           1       0.89      0.87      0.88      1416

    accuracy                           0.83      2000
   macro avg       0.80      0.81      0.80      2000
weighted avg       0.84      0.83      0.84      2000



In [174]:
#sorry that i couldnt include wordcloud
